<h2 style='color:blue'>Small Image Classification Using Simple Aritifical Neural Network: GPU Benchmarking</h2>

In [7]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
# Version Information
# tensorflow 2.2.0 , Cudnn7.6.5 and Cuda 10.1 , python 3.8

ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)

**This command shows list of physical devices available for tensorflow. You can see GPU listed here. If you have NVIDIA GPU you need to install CUDA toolkit and cuDNN as per instruction on this webpage. Without proper installation you will not see GPU in list of devices**

https://shawnhymel.com/1961/how-to-install-tensorflow-with-gpu-support-on-windows/

In [14]:
import tensorflow
print(tensorflow.__file__)

!pip uninstall tensorflow -y
!pip install tensorflow



None
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/375.7 MB 1.4 MB/s eta 0:04:30
   ---------------------------------------- 0.8/375.7 MB 1.6 MB/s eta 0:03:55
   ---------------------------------------- 1.3/375.7 MB 1.7 MB/s eta 0:03:38
   ---------------------------------------- 1.6/375.7 MB 1.7 MB/s eta 0:03:43
   ---------------------------------------- 1.8/375.7 MB 1.6 MB/s eta 0:03:51
   ---------------------------------------- 2.4/375.7 MB 1.7 MB/s eta 0:03:38
   ---------------------------------------- 2.6/375.7 MB 1.7 MB/s eta 0:03:40
   ---------------------------------------- 2.9/375.7 MB 1.6 MB/s eta 0:03:47
   ---------------------------------------- 3.4/375.7 MB 1.7 MB/s eta 0:03:39
   ------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 3.10.0 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.0.2 which is incompatible.
tensorflow-intel 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.19.0 which is incompatible.


In [15]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras


AttributeError: module 'tensorflow' has no attribute '__version__'

In [13]:
tf.config.experimental.list_physical_devices()

AttributeError: module 'tensorflow' has no attribute 'config'

In [ ]:
tf.__version__

'2.4.0'

In [ ]:
tf.test.is_built_with_cuda()

True

<h4 style="color:purple">Load the dataset</h4>

Our dataset contains 60000 small training images that belongs to one of the below 10 classes

<img src="small_images.jpg" />

In [ ]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 31s 0us/step


In [ ]:
X_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 1)

In [ ]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [ ]:
classes[y_train[3][0]]

'deer'

In [ ]:
y_train[:3]

array([[6],
       [9],
       [9]], dtype=uint8)

In [ ]:
y_test.shape

(10000, 1)

In [ ]:
X_train.shape

(50000, 32, 32, 3)

<h4 style="color:purple">Preprocessing: Scale images</h4>

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [ ]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [ ]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [ ]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

<h4 style="color:purple">Model building and training</h4>

In [ ]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [ ]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [ ]:
strategy.num_replicas_in_sync

4

In [ ]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [ ]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
50/50 [==============================] - 3s 9ms/step - loss: 2.1940 - accuracy: 0.2043
Epoch 2/50
50/50 [==============================] - 1s 11ms/step - loss: 1.9763 - accuracy: 0.3150
Epoch 3/50
50/50 [==============================] - 0s 9ms/step - loss: 1.8989 - accuracy: 0.3441
Epoch 4/50
50/50 [==============================] - 0s 9ms/step - loss: 1.8531 - accuracy: 0.3617
Epoch 5/50
50/50 [==============================] - 0s 9ms/step - loss: 1.8211 - accuracy: 0.3709
Epoch 6/50
50/50 [==============================] - 0s 9ms/step - loss: 1.7967 - accuracy: 0.3781
Epoch 7/50
50/50 [==============================] - 0s 9ms/step - loss: 1.7769 - accuracy: 0.3862
Epoch 8/50
50/50 [==============================] - 1s 11ms/step - loss: 1.7599 - accuracy: 0.3928
Epoch 9/50
50/50 [==============================] - 0s 9ms/step - loss: 1.7449 - accuracy: 0.3973
Epoch 10/50
50/50 [==============================] - 1s 11ms/step - loss: 1.7315 - accuracy: 0.4023
Epoch 11/50
50/5

<h3 style='color:purple'>Measure training time on a CPU<h3>

In [ ]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
50/50 [==============================] - 2s 40ms/step - loss: 2.2248 - accuracy: 0.2036
Epoch 2/50
50/50 [==============================] - 2s 40ms/step - loss: 1.9823 - accuracy: 0.3173
Epoch 3/50
50/50 [==============================] - 2s 41ms/step - loss: 1.9027 - accuracy: 0.3438
Epoch 4/50
50/50 [==============================] - 2s 42ms/step - loss: 1.8553 - accuracy: 0.3602
Epoch 5/50
50/50 [==============================] - 2s 40ms/step - loss: 1.8222 - accuracy: 0.3708
Epoch 6/50
50/50 [==============================] - 2s 40ms/step - loss: 1.7968 - accuracy: 0.3789
Epoch 7/50
50/50 [==============================] - 2s 40ms/step - loss: 1.7760 - accuracy: 0.3863
Epoch 8/50
50/50 [==============================] - 2s 41ms/step - loss: 1.7583 - accuracy: 0.3926
Epoch 9/50
50/50 [==============================] - 2s 39ms/step - loss: 1.7428 - accuracy: 0.3989
Epoch 10/50
50/50 [==============================] - 2s 40ms/step - loss: 1.7291 - accuracy: 0.4045
Epoch 11/

So comparing DGX A100 GPU performance with its CPU we get following numbers,

| Epoch | CPU | GPU  |
|:------|:------|:------|
| 50 | 1 min 57 sec | 47 sec |
